In [ ]:
# this code gets 10 levels of orderbook data every second from IBKR, must be run in live notebook during market hours, saved to a csv file on macbook desktop

In [3]:
import os
import time
from ib_insync import *
import pandas as pd
import random

# Connect IB once synchronously; IB will run its own event loop internally
ib = IB()
client_id = random.randint(1, 10000)
print(f'Connecting with client ID: {client_id}')
ib.connect('127.0.0.1', 7497, clientId=client_id)

# Define the contract for Tesla on NASDAQ
contract = Stock('TSLA', 'NASDAQ', 'USD')

# Specify number of depth rows per side (max 10 levels)
num_levels = 10

# Subscribe to market depth data (Level 2) with 10 levels per side
ticker = ib.reqMktDepth(contract, numRows=num_levels)

# Prepare CSV file path on desktop
desktop_path = os.path.join(os.path.expanduser('~'), 'Desktop')
csv_path = os.path.join(desktop_path, 'tesla realtime l2.csv')

# Write header for CSV
with open(csv_path, 'w') as f:
    f.write('timestamp,side,position,operation,price,size\n')

print('Starting to collect order book data every second...')

try:
    while True:
        # Collect current bid and ask depth data snapshot
        rows = []
        timestamp = pd.Timestamp.now()
        for bid in ticker.domBids:
            rows.append([timestamp, 'bid', bid.position, bid.operation, bid.price, bid.size])
        for ask in ticker.domAsks:
            rows.append([timestamp, 'ask', ask.position, ask.operation, ask.price, ask.size])

        # Append rows to CSV
        df = pd.DataFrame(rows, columns=['timestamp', 'side', 'position', 'operation', 'price', 'size'])
        df.to_csv(csv_path, mode='a', header=False, index=False)

        print(f"Appended {len(rows)} rows at {timestamp}")

        time.sleep(1)  # Wait 1 second before next snapshot

except KeyboardInterrupt:
    print('Stopping data collection...')

finally:
    ib.cancelMktDepth(contract)
    ib.disconnect()
    print('Disconnected from IB API')


Connecting with client ID: 6404
Starting to collect order book data every second...
Appended 0 rows at 2025-11-29 21:23:03.054249
Appended 0 rows at 2025-11-29 21:23:04.066337
Appended 0 rows at 2025-11-29 21:23:05.070372
Appended 0 rows at 2025-11-29 21:23:06.078677
Appended 0 rows at 2025-11-29 21:23:07.085372
Appended 0 rows at 2025-11-29 21:23:08.087786
Stopping data collection...
Disconnected from IB API
